In [1]:
import pandas as pd

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/myTransactions.csv',encoding= 'unicode_escape')

stock_ids = mypf[mypf['type'] == 'stock']['id'].unique()

In [2]:
stock_ids

array(['3iinfoltd', 'alkylamine', 'asianpaint', 'dmart', 'axisbank',
       'bajfinance', 'bergepaint', 'bhartiartl', 'canbk', 'divislab',
       'federalbnk', 'havells', 'hcltech', 'hdfcbank', 'hdfclife',
       'heromotoco', 'hindunilvr', 'icicibank', 'ibrealest', 'iex',
       'irctc', 'indusindbk', 'naukri', 'infy', 'kelltontec', 'mhril',
       'mindtree', 'motilalofs', 'nationalum', 'paras', 'precwire',
       'reliance', 'sail', 'suzlon', 'tcs', 'tataconsum', 'tatamotors',
       'tatapower', 'tatasteel', 'tipsindltd', 'titan', 'zeel'],
      dtype=object)

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df_analysis = pd.DataFrame()

for stock_name in stock_ids: 

#stock_name = 'tcs'

    print(stock_name)

    link_blueprint = "https://www.screener.in/company/"
    
    actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"
    
    def get_list(actual_link):
        page = requests.get(actual_link)
        link_soup = BeautifulSoup(page.content, 'html.parser')
        
        test_list = link_soup.find(id="quarters").text.replace("  ", "").split('\n')

        while("" in test_list) :
            test_list.remove("")
        while("\r" in test_list) :
            test_list.remove("\r")        
        while(" \r" in test_list) :
            test_list.remove(" \r")
            
        return test_list
    
    test_list = get_list(actual_link)
        
    print(len(test_list))
    n = len(test_list) 
    
    if n < 114:
        actual_link = link_blueprint + stock_name.upper() 
        test_list = get_list(actual_link)
        print(len(test_list))
    
    q = 0
        
    if n > 162:
        p = 8
        q = n-162
    elif n > 120 and n <= 162:
        p = 8
    elif n < 120 and n > 100:
        p = 7   
        
    else:
        p = 6

    df_qreults = pd.DataFrame()
    
    if 'Gross NPA %'  in test_list:
        endkey = 'Gross NPA %'
    else:
        endkey = 'Raw PDF'

    if 'Dec 2021' in test_list:
        quarter = 'Dec 2021'
    else:
        quarter = 'Sep 2021'

    #df_qreults['Quarter'] = test_list[test_list.index('Sep 2021')-p:test_list.index('Sep 2021')+1]
    #df_qreults['EPS(Q)'] = test_list[test_list.index('Raw PDF')-p-q-1:test_list.index('Raw PDF')-q]
    s = test_list.index(endkey)  - test_list.index('EPS in Rs')-2
    df_qreults['Quarter'] = test_list[test_list.index(quarter)-s:test_list.index(quarter)+1]    
    df_qreults['EPS(Q)'] = test_list[test_list.index('EPS in Rs')+1:test_list.index(endkey)]

    try:
        df_qreults['Sales(Q)'] = test_list[test_list.index('Expenses\xa0+')-s-1:test_list.index('Expenses\xa0+')]
    except:
        df_qreults['Sales(Q)'] = test_list[test_list.index('Interest')-s-1:test_list.index('Interest')] 
    
    
    df_qreults['Sales(Q)']  = df_qreults['Sales(Q)'].str.replace(",", "")

    df_qreults['EPS(Q)'] = df_qreults['EPS(Q)'].astype(float).fillna(0.0)
    df_qreults['Sales(Q)'] = df_qreults['Sales(Q)'].astype(float).fillna(0.0)

    from pandas.tseries.offsets import MonthEnd
    import datetime

    df_qreults['Quarter'] = pd.to_datetime(df_qreults['Quarter'], format="%b %Y") + MonthEnd(1) + datetime.timedelta(days=46)
    df_qreults

    df_qreults['EPS(TTM)'] = df_qreults['EPS(Q)'] + df_qreults['EPS(Q)'].shift(1) + df_qreults['EPS(Q)'].shift(2) + \
                                df_qreults['EPS(Q)'].shift(3)
    
    df_qreults['Sales(TTM)'] = df_qreults['Sales(Q)'] + df_qreults['Sales(Q)'].shift(1) + df_qreults['Sales(Q)'].shift(2) + \
                                df_qreults['Sales(Q)'].shift(3)

    import pandas_datareader.data as web
    import yfinance as yfin
    yfin.pdr_override()

    df_stock = pd.DataFrame()

    for date in df_qreults['Quarter']:
        print(date)

        end_date = date
        start_date = (date + datetime.timedelta(days=-5)).strftime('%Y-%m-%d')


        stock_id = stock_name.upper() + '.NS'
        tmp = pd.DataFrame(web.DataReader(tickers = stock_id, data_source='yahoo', start = start_date, end = end_date))
        tmp = tmp.tail(1)
        tmp['Stock'] = stock_id
        tmp['Quarter'] = end_date  
        tmp['Close'] = round(tmp['Close'],2)
        tmp.reset_index(inplace = True)
        cols = ['Stock','Quarter','Close']
        tmp =tmp[cols]    
        df_stock = df_stock.append(tmp)

    df_stock_analysis = df_stock.merge(df_qreults, on='Quarter').tail(6)
    df_stock_analysis['PE(TTM)'] = round(df_stock_analysis['Close']/df_stock_analysis['EPS(TTM)'],2)
    
    df_analysis = df_analysis.append(df_stock_analysis)
    
df_analysis

3iinfoltd
162
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM)
6,3IINFOLTD.NS,2020-08-15,28.50,0.06,150.0,0.46,891.0,61.96
7,3IINFOLTD.NS,2020-11-15,32.00,0.09,152.0,0.47,747.0,68.09
8,3IINFOLTD.NS,2021-02-15,69.00,0.14,158.0,0.44,618.0,156.82
9,3IINFOLTD.NS,2021-05-16,92.50,2.12,149.0,2.41,609.0,38.38
10,3IINFOLTD.NS,2021-08-15,79.50,-0.05,160.0,2.30,619.0,34.57
...,...,...,...,...,...,...,...,...
7,ZEEL.NS,2020-11-15,193.10,0.98,1723.0,-3.04,7035.0,-63.52
8,ZEEL.NS,2021-02-15,212.50,4.16,2729.0,-2.52,7715.0,-84.33
9,ZEEL.NS,2021-05-16,191.60,2.87,1966.0,8.33,7730.0,23.00
10,ZEEL.NS,2021-08-15,183.25,2.23,1775.0,10.24,8193.0,17.90


In [4]:
df_analysis_final = pd.DataFrame()
for stock_id in df_analysis['Stock'].unique():
    tmp = df_analysis[df_analysis['Stock'] == stock_id]
    tmp['Prev_EPS(Q)'] = tmp['EPS(Q)'].shift(1)
    tmp['Prev_EPS(TTM)'] = tmp['EPS(TTM)'].shift(1)
    tmp['Prev_Sales'] = tmp['Sales(Q)'].shift(1)
    tmp['Prev_PE'] = tmp['PE(TTM)'].shift(1)
    tmp['Prev_Close'] = tmp['Close'].shift(1)
    tmp['FR%'] = round((tmp['EPS(TTM)'] - tmp['Prev_EPS(TTM)'])*100/tmp['Prev_EPS(TTM)'],2)
    tmp['SR%'] = round(tmp['EPS(TTM)']*(tmp['PE(TTM)'] - tmp['Prev_PE'])*100/tmp['Prev_Close'],2)
    tmp['TR%'] = tmp['FR%'] + tmp['SR%']
    tmp['Sales%'] = round((tmp['Sales(Q)'] - tmp['Prev_Sales'])*100/tmp['Prev_Sales'],2)
    tmp['EPS%'] = round((tmp['EPS(Q)'] - tmp['Prev_EPS(Q)'])*100/tmp['Prev_EPS(Q)'],2)
    tmp['Return%'] = tmp['FR%'] + tmp['SR%']
    df_analysis_final = df_analysis_final.append(tmp)
    
df_analysis_final

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM),Prev_EPS(Q),Prev_EPS(TTM),Prev_Sales,Prev_PE,Prev_Close,FR%,SR%,TR%,Sales%,EPS%,Return%
6,3IINFOLTD.NS,2020-08-15,28.50,0.06,150.0,0.46,891.0,61.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,3IINFOLTD.NS,2020-11-15,32.00,0.09,152.0,0.47,747.0,68.09,0.06,0.46,150.0,61.96,28.50,2.17,10.11,12.28,1.33,50.00,12.28
8,3IINFOLTD.NS,2021-02-15,69.00,0.14,158.0,0.44,618.0,156.82,0.09,0.47,152.0,68.09,32.00,-6.38,122.00,115.62,3.95,55.56,115.62
9,3IINFOLTD.NS,2021-05-16,92.50,2.12,149.0,2.41,609.0,38.38,0.14,0.44,158.0,156.82,69.00,447.73,-413.68,34.05,-5.70,1414.29,34.05
10,3IINFOLTD.NS,2021-08-15,79.50,-0.05,160.0,2.30,619.0,34.57,2.12,2.41,149.0,38.38,92.50,-4.56,-9.47,-14.03,7.38,-102.36,-14.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,ZEEL.NS,2020-11-15,193.10,0.98,1723.0,-3.04,7035.0,-63.52,0.32,0.28,1312.0,575.71,161.20,-1185.71,1205.50,19.79,31.33,206.25,19.79
8,ZEEL.NS,2021-02-15,212.50,4.16,2729.0,-2.52,7715.0,-84.33,0.98,-3.04,1723.0,-63.52,193.10,-17.11,27.16,10.05,58.39,324.49,10.05
9,ZEEL.NS,2021-05-16,191.60,2.87,1966.0,8.33,7730.0,23.00,4.16,-2.52,2729.0,-84.33,212.50,-430.56,420.73,-9.83,-27.96,-31.01,-9.83
10,ZEEL.NS,2021-08-15,183.25,2.23,1775.0,10.24,8193.0,17.90,2.87,8.33,1966.0,23.00,191.60,22.93,-27.26,-4.33,-9.72,-22.30,-4.33


In [5]:
cols = ['Stock','Quarter','Sales%','EPS%','Return%']
df_growth = df_analysis_final[cols]
df_growth[df_growth['Quarter'] == '2021-11-15'].sort_values(by='Sales%', ascending=False).head(20)

,Stock,Quarter,Sales%,EPS%,Return%
11,TIPSINDLTD.NS,2021-11-15,242.86,44.22,26.53
11,TITAN.NS,2021-11-15,115.75,3030.43,38.17
8,IRCTC.NS,2021-11-15,66.67,92.23,62.52
11,HEROMOTOCO.NS,2021-11-15,55.17,191.11,-3.22
10,DMART.NS,2021-11-15,50.28,338.78,41.29
11,MOTILALOFS.NS,2021-11-15,47.39,142.36,17.78
11,MHRIL.NS,2021-11-15,47.17,-374.77,21.36
11,NATIONALUM.NS,2021-11-15,45.13,115.34,21.24
11,HDFCLIFE.NS,2021-11-15,38.97,2.26,7.47
11,PRECWIRE.NS,2021-11-15,33.52,29.00,38.41


In [6]:
#df_analysis_final.to_csv('data/Portfolio_FvS_QoQ_Analysis.csv', index=False)

In [7]:
#df_growth = pd.read_csv('data/Portfolio_FvS_QoQ_Analysis.csv')
df_growth = df_analysis_final
cols = ['Stock','Quarter','Sales%','EPS%','Return%']
#df_growth = df_growth[cols]
df_growth = df_growth.dropna()
df_growth

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM),Prev_EPS(Q),Prev_EPS(TTM),Prev_Sales,Prev_PE,Prev_Close,FR%,SR%,TR%,Sales%,EPS%,Return%
7,3IINFOLTD.NS,2020-11-15,32.00,0.09,152.0,0.47,747.0,68.09,0.06,0.46,150.0,61.96,28.50,2.17,10.11,12.28,1.33,50.00,12.28
8,3IINFOLTD.NS,2021-02-15,69.00,0.14,158.0,0.44,618.0,156.82,0.09,0.47,152.0,68.09,32.00,-6.38,122.00,115.62,3.95,55.56,115.62
9,3IINFOLTD.NS,2021-05-16,92.50,2.12,149.0,2.41,609.0,38.38,0.14,0.44,158.0,156.82,69.00,447.73,-413.68,34.05,-5.70,1414.29,34.05
10,3IINFOLTD.NS,2021-08-15,79.50,-0.05,160.0,2.30,619.0,34.57,2.12,2.41,149.0,38.38,92.50,-4.56,-9.47,-14.03,7.38,-102.36,-14.03
11,3IINFOLTD.NS,2021-11-15,70.05,-1.43,177.0,0.78,644.0,89.81,-0.05,2.30,160.0,34.57,79.50,-66.09,54.20,-11.89,10.62,2760.00,-11.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,ZEEL.NS,2020-11-15,193.10,0.98,1723.0,-3.04,7035.0,-63.52,0.32,0.28,1312.0,575.71,161.20,-1185.71,1205.50,19.79,31.33,206.25,19.79
8,ZEEL.NS,2021-02-15,212.50,4.16,2729.0,-2.52,7715.0,-84.33,0.98,-3.04,1723.0,-63.52,193.10,-17.11,27.16,10.05,58.39,324.49,10.05
9,ZEEL.NS,2021-05-16,191.60,2.87,1966.0,8.33,7730.0,23.00,4.16,-2.52,2729.0,-84.33,212.50,-430.56,420.73,-9.83,-27.96,-31.01,-9.83
10,ZEEL.NS,2021-08-15,183.25,2.23,1775.0,10.24,8193.0,17.90,2.87,8.33,1966.0,23.00,191.60,22.93,-27.26,-4.33,-9.72,-22.30,-4.33


In [8]:
df_growth[(df_growth['Sales%'] > 5) & (df_growth['EPS%'] > 5)]['Stock'].value_counts()[0:20]

TATASTEEL.NS     4
MOTILALOFS.NS    4
IRCTC.NS         3
ALKYLAMINE.NS    3
TIPSINDLTD.NS    3
ZEEL.NS          3
HAVELLS.NS       3
NATIONALUM.NS    3
MINDTREE.NS      3
DMART.NS         3
PRECWIRE.NS      3
SAIL.NS          3
BERGEPAINT.NS    3
TITAN.NS         2
DIVISLAB.NS      2
TATAPOWER.NS     2
NAUKRI.NS        2
INFY.NS          2
HEROMOTOCO.NS    2
ASIANPAINT.NS    2
Name: Stock, dtype: int64

In [9]:
df_tmp_final = pd.DataFrame()
for stock in df_growth['Stock'].unique():
    df_tmp = df_growth[df_growth['Stock'] == stock][cols]
    df_tmp['sales_flag'] = 0
    df_tmp['eps_flag'] = 0
    df_tmp.loc[df_tmp['Sales%'] < 0, 'sales_flag'] = 1
    df_tmp.loc[df_tmp['EPS%'] < 0, 'eps_flag'] = 1
    df_tmp['sales_weight'] =  df_tmp['sales_flag']*5 + df_tmp['sales_flag'].shift(1)*4 + df_tmp['sales_flag'].shift(2)*3+df_tmp['sales_flag'].shift(3)*2+df_tmp['sales_flag'].shift(4)*1
    df_tmp['eps_weight'] =  df_tmp['eps_flag']*5 + df_tmp['eps_flag'].shift(1)*4 + df_tmp['eps_flag'].shift(2)*3+df_tmp['eps_flag'].shift(3)*2+df_tmp['eps_flag'].shift(4)*1
    df_tmp['total_weight'] = df_tmp['sales_weight'] + df_tmp['eps_weight'] 
    df_tmp = df_tmp.tail(1)
    df_tmp_final = df_tmp_final.append(df_tmp)

In [10]:
df_tmp_final[(df_tmp_final['Sales%'] > 5) & (df_tmp_final['EPS%'] > 5)].sort_values(by = 'total_weight', ascending=True)

,Stock,Quarter,Sales%,EPS%,Return%,sales_flag,eps_flag,sales_weight,eps_weight,total_weight
11,TATASTEEL.NS,2021-11-15,12.95,33.73,-11.99,0,0,0.0,1.0,1.0
10,INFY.NS,2021-11-15,6.12,5.66,3.97,0,0,0.0,3.0,3.0
10,MINDTREE.NS,2021-11-15,12.83,16.12,68.22,0,0,0.0,3.0,3.0
11,IEX.NS,2021-11-15,20.88,22.86,93.71,0,0,4.0,0.0,4.0
11,SAIL.NS,2021-11-15,29.96,11.23,-12.47,0,0,4.0,1.0,5.0
11,TIPSINDLTD.NS,2021-11-15,242.86,44.22,26.53,0,0,3.0,4.0,7.0
11,3IINFOLTD.NS,2021-11-15,10.62,2760.00,-11.89,0,0,3.0,4.0,7.0
11,NAUKRI.NS,2021-11-15,11.31,4738.29,18.12,0,0,1.0,7.0,8.0
11,MOTILALOFS.NS,2021-11-15,47.39,142.36,17.78,0,0,4.0,4.0,8.0
11,NATIONALUM.NS,2021-11-15,45.13,115.34,21.24,0,0,4.0,4.0,8.0


In [11]:
df_growth[df_growth['Stock'] == 'HDFCBANK.NS'][cols]

,Stock,Quarter,Sales%,EPS%,Return%
6,HDFCBANK.NS,2020-11-15,-3.68,10.94,32.40
7,HDFCBANK.NS,2021-02-15,-3.39,13.71,15.48
8,HDFCBANK.NS,2021-05-16,-3.03,-3.89,-12.32
9,HDFCBANK.NS,2021-08-15,0.96,-6.34,10.04
10,HDFCBANK.NS,2021-11-15,1.39,14.65,1.77


In [12]:
df_growth[df_growth['Stock'] == 'ICICIBANK.NS'][cols]

,Stock,Quarter,Sales%,EPS%,Return%
7,ICICIBANK.NS,2020-11-15,-3.54,47.19,34.75
8,ICICIBANK.NS,2021-02-15,-4.87,12.43,33.00
9,ICICIBANK.NS,2021-05-16,-4.39,-11.18,-7.74
10,ICICIBANK.NS,2021-08-15,1.14,-2.69,17.98
11,ICICIBANK.NS,2021-11-15,0.25,27.62,10.17


In [13]:
df_growth[df_growth['Stock'] == 'AXISBANK.NS'][cols]

,Stock,Quarter,Sales%,EPS%,Return%
7,AXISBANK.NS,2020-11-15,-8.44,53.85,40.69
8,AXISBANK.NS,2021-02-15,-6.02,-28.33,22.38
9,AXISBANK.NS,2021-05-16,-2.12,123.26,-8.72
10,AXISBANK.NS,2021-08-15,3.59,-19.90,10.93
11,AXISBANK.NS,2021-11-15,2.76,43.69,-3.13


In [14]:
df_growth[df_growth['Stock'] == 'INDUSINDBK.NS'][cols]

,Stock,Quarter,Sales%,EPS%,Return%
7,INDUSINDBK.NS,2020-11-15,1.19,19.16,51.91
8,INDUSINDBK.NS,2021-02-15,-1.64,25.09,33.34
9,INDUSINDBK.NS,2021-05-16,1.30,9.21,-13.26
10,INDUSINDBK.NS,2021-08-15,3.24,9.60,14.96
11,INDUSINDBK.NS,2021-11-15,-0.47,12.80,1.25


In [15]:
df_tmp_final.sort_values(by = 'total_weight', ascending=True)

,Stock,Quarter,Sales%,EPS%,Return%,sales_flag,eps_flag,sales_weight,eps_weight,total_weight
11,TATASTEEL.NS,2021-11-15,12.95,33.73,-11.99,0,0,0.0,1.0,1.0
10,INFY.NS,2021-11-15,6.12,5.66,3.97,0,0,0.0,3.0,3.0
10,HCLTECH.NS,2021-11-15,2.93,1.69,4.74,0,0,0.0,3.0,3.0
10,MINDTREE.NS,2021-11-15,12.83,16.12,68.22,0,0,0.0,3.0,3.0
11,IEX.NS,2021-11-15,20.88,22.86,93.71,0,0,4.0,0.0,4.0
10,TCS.NS,2021-11-15,3.21,6.86,1.82,0,0,0.0,4.0,4.0
11,DIVISLAB.NS,2021-11-15,1.38,8.81,-2.12,0,0,2.0,2.0,4.0
11,SAIL.NS,2021-11-15,29.96,11.23,-12.47,0,0,4.0,1.0,5.0
11,3IINFOLTD.NS,2021-11-15,10.62,2760.00,-11.89,0,0,3.0,4.0,7.0
11,TIPSINDLTD.NS,2021-11-15,242.86,44.22,26.53,0,0,3.0,4.0,7.0
